Getting the energy functions to work

In [1]:
type LatticeSite
    A::Array{Float64,1}  # Fluctuating vector potential
    θ⁺::Float64 # Phase of the + component
    θ⁻::Float64 # Phase of the - component
    u⁺::Float64 # Amplitude of + component
end
type State
    lattice::Array{LatticeSite,2}  # Numerical lattice
    γ::Float64    # Order parameter amplitude
    g::Float64    # Gauge coupling
    ν::Float64    # Anisotropy constant
    f::Float64    # Magnetic filling fraction
end
function initializeState(N::Int64, choice::Int64)
    N <= 1 && throw(DomainError())
    
    # Constants
    γ = 1.0    # Order parameter amplitude
    g = 1.0    # Gauge coupling
    ν = 0.0    # Anisotropy constant
    f = 0.0    # Magnetic filling fraction
    
    # Construct ordered state 
    if choice == 1
        
        # Construct NxN lattice of NxN LatticeSites
        lattice = [LatticeSite([0,0],0,0,0) for y=1:N, x=1:N]
        ψ = State(lattice, γ, g, ν, f)
        
    # Construct random state
    elseif choice == 2
        Amax::Int64 = 2^10
        lattice = [LatticeSite([rand()*Amax*(-1)^rand(0:1),rand()*Amax*(-1)^rand(0:1)],
                rand()*2*π,rand()*2*π, rand()) for y=1:N, x=1:N]
        ψ = State(lattice, γ, g, ν, f)
        
    # We only have choices 1 and 2 so far so other values for choice will give an error.
    else
        throw(DomainError())
    end
    ψ
end
import Base.copy
function copy(ϕ::LatticeSite)
    LatticeSite([ϕ.A[1],ϕ.A[2]],ϕ.θ⁺,ϕ.θ⁻,ϕ.u⁺)
end
function copy(ψ::State)
    Lx = size(ψ.lattice,2)
    Ly = size(ψ.lattice,1)
    lattice = [LatticeSite([ψ.lattice[y,x].A[1],ψ.lattice[y,x].A[2]],ψ.lattice[y,x].θ⁺,ψ.lattice[y,x].θ⁻,
            ψ.lattice[y,x].u⁺) for y = 1:Ly, x=1:Lx]
    State(lattice, ψ.γ, ψ.g, ψ.ν, ψ.f)
end

copy (generic function with 46 methods)

In [2]:
function fᵣ(ϕ::LatticeSite,ϕᵣ₊₁::LatticeSite,ϕᵣ₊₂::LatticeSite,A₀::Float64,γ::Float64,g::Float64,ν::Float64,f::Float64)
    energy = 0
    uᵣ₊₁⁻ = √(1-ϕᵣ₊₁.u⁺^2)
    u⁻ = √(1-ϕ.u⁺^2)
    uᵣ₊₂⁻ = √(1-ϕᵣ₊₂.u⁺^2)
    A₂ = ϕ.A[2]+A₀
    # Kinetic energy Fₖ
    energy += -2*γ^2*(ϕᵣ₊₁.u⁺ *ϕ.u⁺*cos(ϕᵣ₊₁.θ⁺-ϕ.θ⁺ + g*ϕ.A[1]) 
        + ϕᵣ₊₂.u⁺*ϕ.u⁺*cos(ϕᵣ₊₂.θ⁺-ϕ.θ⁺ + g*A₂) 
        + uᵣ₊₁⁻*u⁻*cos(ϕᵣ₊₁.θ⁻-ϕ.θ⁻ + g*ϕ.A[1]) 
        + uᵣ₊₂⁻*u⁻*cos(ϕᵣ₊₂.θ⁻-ϕ.θ⁻ + g*A₂) )
    # Potential energy Fᵥ
    energy += γ^4*ϕ.u⁺^2*u⁻^2*(1+ν*cos(2*(ϕ.θ⁺-ϕ.θ⁻)))
    # Andreev Bashkin terms
    energy += γ^2*(ν+1)*(u⁻*ϕᵣ₊₂.u⁺*cos(ϕᵣ₊₂.θ⁺-ϕ.θ⁻ + g*A₂) 
        + ϕ.u⁺*uᵣ₊₂⁻*cos(ϕᵣ₊₂.θ⁻-ϕ.θ⁺ + g*A₂) 
        - u⁻*ϕᵣ₊₁.u⁺*cos(ϕᵣ₊₁.θ⁺ - ϕ.θ⁻ + g*ϕ.A[1]) 
        - ϕ.u⁺*uᵣ₊₁⁻*cos(ϕᵣ₊₁.θ⁻ - ϕ.θ⁺ + g*ϕ.A[1]))
    # Mixed gradient terms
    energy += γ^2*(ν-1)*(ϕᵣ₊₂.u⁺*uᵣ₊₁⁻*sin(ϕᵣ₊₁.θ⁻ - ϕᵣ₊₂.θ⁺ + g*(ϕ.A[1]-A₂)) 
        - uᵣ₊₂⁻*ϕᵣ₊₁.u⁺*sin(ϕᵣ₊₁.θ⁺ - ϕᵣ₊₂.θ⁻ + g*(ϕ.A[1] - A₂)) 
        + 2*ϕ.u⁺*u⁻*sin(ϕ.θ⁻-ϕ.θ⁺) 
        -  u⁻*ϕᵣ₊₂.u⁺*sin(ϕᵣ₊₂.θ⁺-ϕ.θ⁻ + g*A₂) 
        + ϕ.u⁺*uᵣ₊₂⁻*sin(ϕᵣ₊₂.θ⁻-ϕ.θ⁺ + g*A₂) 
        + u⁻*ϕᵣ₊₁.u⁺*sin(ϕᵣ₊₁.θ⁺ - ϕ.θ⁻ + g*ϕ.A[1]) 
        - ϕ.u⁺*uᵣ₊₁⁻*sin(ϕᵣ₊₁.θ⁻ - ϕ.θ⁺ + g*ϕ.A[1]))
    energy
end

fᵣ (generic function with 1 method)

In [3]:
# Loops over all positions of the lattice of a state and calculates the total energy from the
# Higgs-field terms using the function fᵣ() + the energy from the gauge field.
function E(ψ::State)
    γ = ψ.γ
    g = ψ.g
    energy = 0.0
    ν = ψ.ν
    f = ψ.f
    Lx = size(ψ.lattice,2)
    Ly = size(ψ.lattice,1)
    
    # Contribution from upper right corner
    A⁰ = (Lx-1)*2π*f
    ϕ = ψ.lattice[1,Lx]    # Lattice site at upper right corner
    ϕᵣ₊₁ = ψ.lattice[1,1]    # Nearest neighbor at r+x is upper left corner
    ϕᵣ₊₂ = ψ.lattice[Ly,Lx]  # Nearest neighbor at r+y is lower right corner
    energy += fᵣ(ϕ,ϕᵣ₊₁,ϕᵣ₊₂,A⁰,γ,g,ν,f)              # Higgs terms
    energy += (ϕ.A[1] + ϕᵣ₊₁.A[2]-ϕᵣ₊₂.A[1]-ϕ.A[2])^2 # Maxwell term
    
    # Contribution from right boundary paralell to y-axis
    # except for the upper right corneϕ.
    for y=2:Ly
        ϕ = ψ.lattice[y,Lx]
        ϕᵣ₊₁ = ψ.lattice[y,1]
        ϕᵣ₊₂ = ψ.lattice[y-1,Lx]
        A⁰ = (Lx-1)*2*π*f
        energy += fᵣ(ϕ,ϕᵣ₊₁,ϕᵣ₊₂,A⁰,γ,g,ν,f)              # Higgs terms
        energy += (ϕ.A[1] + ϕᵣ₊₁.A[2]-ϕᵣ₊₂.A[1]-ϕ.A[2])^2
    end
    
    # Contribution from the bulk of lattice sites and upper boundary
    for x=1:(Lx-1)
        A⁰ = (x-1)*2*π*f        # Constant vector potential.
        # Constribution from upper boundary except upper right corner
        ϕ = ψ.lattice[1,x]
        ϕᵣ₊₁ = ψ.lattice[1,x+1]
        ϕᵣ₊₂ = ψ.lattice[Ly,x]
        energy += fᵣ(ϕ,ϕᵣ₊₁,ϕᵣ₊₂,A⁰,γ,g,ν,f)              # Higgs terms
        energy += (ϕ.A[1] + ϕᵣ₊₁.A[2]-ϕᵣ₊₂.A[1]-ϕ.A[2])^2
        
        # Contribution from the rest of the bulk.
        for y=2:Ly
            ϕ = ψ.lattice[y,x]          # Lattice site at position r
            ϕᵣ₊₁ = ψ.lattice[y,x+1]       # Nearest neighbor at r+x
            ϕᵣ₊₂ = ψ.lattice[y-1,x]       # Nearest neighbor at r+y
            
            energy += fᵣ(ϕ,ϕᵣ₊₁,ϕᵣ₊₂,A⁰,γ,g,ν,f)              # Higgs terms
            energy += (ϕ.A[1] + ϕᵣ₊₁.A[2]-ϕᵣ₊₂.A[1]-ϕ.A[2])^2
        end
    end
    
    energy
end

E (generic function with 1 method)

Let's do a unit test that the energy function acts as expected.

In [ ]:
# Tests mostly that the kinetic energy Fₖ is correctly calculated.
using Base.Test
f = 1.0/33
ψ = initializeState(2, 1)
@test E(ψ) == -4*(3+cos(2*π*1/32))

In [4]:
# If we don't try to be fancy about it, we could just use fᵣ for each of the points around the
# updated lattice site.
#
function ΔE2(ψ::State, ϕ′::LatticeSite, pos::Array{Int64,1})
    x = pos[1]
    y = pos[2]
    Ly = size(ψ.lattice,1)
    Lx = size(ψ.lattice,2)
    δE = 0.0
    ϕ = ψ.lattice[y,x]
    
    # Calculate nearest neighbor lattice sites.
    const ϕᵣ₊₁ = ψ.lattice[y, mod(x,Lx)+1]
    const ϕᵣ₊₂ = ψ.lattice[mod(y-2,Ly)+1,x]
    const ϕᵣ₋₁ = ψ.lattice[y,mod(x-2,Lx)+1]
    const ϕᵣ₋₂ = ψ.lattice[mod(y,Ly)+1,x]
    const ϕᵣ₋₁₊₂ = ψ.lattice[mod(y-2,Ly)+1,mod(x-2,Lx)+1]
    const ϕᵣ₋₂₊₁ = ψ.lattice[mod(y,Ly)+1,mod(x,Lx)+1]
    
    # Energy difference from Higgs field terms
    # First contribution from current position.
    δE += fᵣ(ϕ′,ϕᵣ₊₁,ϕᵣ₊₂,2π*ψ.f*(x-1),ψ.γ,ψ.g,ψ.ν,ψ.f)-fᵣ(ϕ,ϕᵣ₊₁,ϕᵣ₊₂,2π*ψ.f*(x-1),ψ.γ,ψ.g,ψ.ν,ψ.f)
    # Then from position r-x
    δE += fᵣ(ϕᵣ₋₁,ϕ′,ϕᵣ₋₁₊₂,2π*ψ.f*(x-2),ψ.γ,ψ.g,ψ.ν,ψ.f)-fᵣ(ϕᵣ₋₁,ϕ,ϕᵣ₋₁₊₂,2π*ψ.f*(x-2),ψ.γ,ψ.g,ψ.ν,ψ.f)
    # Then from position r-y
    δE += fᵣ(ϕᵣ₋₂,ϕᵣ₋₂₊₁,ϕ′,2π*ψ.f*(x-1),ψ.γ,ψ.g,ψ.ν,ψ.f)-fᵣ(ϕᵣ₋₂,ϕᵣ₋₂₊₁,ϕ,2π*ψ.f*(x-1),ψ.γ,ψ.g,ψ.ν,ψ.f)
    
    # Then calculate the Gauge field contribution
    # First contribution from current position
    δE += (ϕ′.A[1] + ϕᵣ₊₁.A[2] - ϕᵣ₊₂.A[1] - ϕ′.A[2])^2 - (ϕ.A[1] + ϕᵣ₊₁.A[2] - ϕᵣ₊₂.A[1] - ϕ.A[2])^2
    # Then from position r-x
    δE += (ϕᵣ₋₁.A[1] + ϕ′.A[2] - ϕᵣ₋₁₊₂.A[1] - ϕᵣ₋₁.A[2])^2 - (ϕᵣ₋₁.A[1] + ϕ.A[2] - ϕᵣ₋₁₊₂.A[1] - ϕᵣ₋₁.A[2])^2
    # Then from position r-y
    δE += (ϕᵣ₋₂.A[1] + ϕᵣ₋₂₊₁.A[2] - ϕ′.A[1] - ϕᵣ₋₂.A[2])^2 - (ϕᵣ₋₂.A[1] + ϕᵣ₋₂₊₁.A[2] - ϕ.A[1] - ϕᵣ₋₂.A[2])^2
end 

ΔE2 (generic function with 1 method)

In [5]:
# Now we try to be fancy about it and use the reduced formula for the difference between the energies
#
function ΔE3(ψ::State, ϕ′::LatticeSite, pos::Array{Int64,1})
    const x = pos[1]
    const y = pos[2]
    const Ly = size(ψ.lattice,1)
    const Lx = size(ψ.lattice,2)
    const ϕ = ψ.lattice[y,x]
    const g = ψ.g
    const δE::Float64 = 0.0
    
    # Calculate nearest neighbor lattice sites.
    const ϕᵣ₊₁ = ψ.lattice[y, mod(x,Lx)+1]
    const ϕᵣ₊₂ = ψ.lattice[mod(y-2,Ly)+1,x]
    const ϕᵣ₋₁ = ψ.lattice[y,mod(x-2,Lx)+1]
    const ϕᵣ₋₂ = ψ.lattice[mod(y,Ly)+1,x]
    const ϕᵣ₋₁₊₂ = ψ.lattice[mod(y-2,Ly)+1,mod(x-2,Lx)+1]
    const ϕᵣ₋₂₊₁ = ψ.lattice[mod(y,Ly)+1,mod(x,Lx)+1]
    
    # Calculate u⁻ for these lattice sites
    const u′⁻ = √(1-ϕ′.u⁺^2)
    const u⁻ = √(1-ϕ.u⁺^2)
    const u⁻ᵣ₊₁ = √(1-ϕᵣ₊₁.u⁺^2)
    const u⁻ᵣ₊₂ = √(1-ϕᵣ₊₂.u⁺^2)
    const u⁻ᵣ₋₁ = √(1-ϕᵣ₋₁.u⁺^2)
    const u⁻ᵣ₋₂ = √(1-ϕᵣ₋₂.u⁺^2)
    const u⁻ᵣ₋₁₊₂ = √(1-ϕᵣ₋₁₊₂.u⁺^2)
    const u⁻ᵣ₋₂₊₁ = √(1-ϕᵣ₋₂₊₁.u⁺^2)
    
    # Calculate constant link variables
    const A⁰ = 2π*ψ.f*(x-1)
    const A⁰₊ = 2π*ψ.f*x
    const A⁰₋ = 2π*ψ.f*(x-2)
    
    # Normal kinetic terms
    δE += -2*ψ.γ^2*(ϕᵣ₊₁.u⁺*(ϕ′.u⁺*cos(ϕᵣ₊₁.θ⁺ - ϕ′.θ⁺ + g*ϕ′.A[1]) - ϕ.u⁺*cos(ϕᵣ₊₁.θ⁺ - ϕ.θ⁺ + g*ϕ.A[1]))
        + ϕᵣ₋₁.u⁺*(ϕ′.u⁺*cos(ϕ′.θ⁺-ϕᵣ₋₁.θ⁺+g*ϕᵣ₋₁.A[1]) - ϕ.u⁺*cos(ϕ.θ⁺-ϕᵣ₋₁.θ⁺+g*ϕᵣ₋₁.A[1]))
        + u⁻ᵣ₊₁*(u′⁻*cos(ϕᵣ₊₁.θ⁻ - ϕ′.θ⁻ + g*ϕ′.A[1]) - u⁻*cos(ϕᵣ₊₁.θ⁻ - ϕ.θ⁻ + g*ϕ.A[1]))
        + u⁻ᵣ₋₁*(u′⁻*cos(ϕ′.θ⁻-ϕᵣ₋₁.θ⁻+g*ϕᵣ₋₁.A[1]) - u⁻*cos(ϕ.θ⁻-ϕᵣ₋₁.θ⁻+g*ϕᵣ₋₁.A[1]))
        + ϕᵣ₊₂.u⁺*(ϕ′.u⁺*cos(ϕᵣ₊₂.θ⁺ - ϕ′.θ⁺ + g*(ϕ′.A[2]+A⁰)) - ϕ.u⁺*cos(ϕᵣ₊₂.θ⁺ - ϕ.θ⁺ + g*(ϕ.A[2]+A⁰)))
        + ϕᵣ₋₂.u⁺*(ϕ′.u⁺*cos(ϕ′.θ⁺-ϕᵣ₋₂.θ⁺+g*(ϕᵣ₋₂.A[2]+A⁰)) - ϕ.u⁺*cos(ϕ.θ⁺-ϕᵣ₋₂.θ⁺+g*(ϕᵣ₋₂.A[2]+A⁰)))
        + u⁻ᵣ₊₂*(u′⁻*cos(ϕᵣ₊₂.θ⁻ - ϕ′.θ⁻ + g*(ϕ′.A[2]+A⁰)) - u⁻*cos(ϕᵣ₊₂.θ⁻ - ϕ.θ⁻ + g*(ϕ.A[2]+A⁰)))
        + u⁻ᵣ₋₂*(u′⁻*cos(ϕ′.θ⁻-ϕᵣ₋₂.θ⁻+g*(ϕᵣ₋₂.A[2]+A⁰)) - u⁻*cos(ϕ.θ⁻-ϕᵣ₋₂.θ⁻+g*(ϕᵣ₋₂.A[2]+A⁰))))
    
    # Potential energy terms
    δE += ψ.γ^4*((ϕ′.u⁺*u′⁻)^2*(1+ψ.ν*cos(2*(ϕ′.θ⁺ - ϕ′.θ⁻))) - (ϕ.u⁺*u⁻)^2*(1+ψ.ν*cos(2*(ϕ.θ⁺ - ϕ.θ⁻))))
    
    # Andreev-Bashkin terms
    δE += ψ.γ^2*(ψ.ν+1)*(ϕᵣ₊₂.u⁺*(u′⁻*cos(ϕᵣ₊₂.θ⁺-ϕ′.θ⁻+g*(ϕ′.A[2]+A⁰)) - u⁻*cos(ϕᵣ₊₂.θ⁺-ϕ.θ⁻+g*(ϕ.A[2]+A⁰))) 
        - ϕᵣ₊₁.u⁺*(u′⁻*cos(ϕᵣ₊₁.θ⁺-ϕ′.θ⁻+g*ϕ′.A[1]) - u⁻*cos(ϕᵣ₊₁.θ⁺-ϕ.θ⁻+g*ϕ.A[1])) 
        + u⁻ᵣ₋₂*(ϕ′.u⁺*cos(ϕ′.θ⁺-ϕᵣ₋₂.θ⁻+g*(ϕᵣ₋₂.A[2]+A⁰)) - ϕ.u⁺*cos(ϕ.θ⁺-ϕᵣ₋₂.θ⁻+g*(ϕᵣ₋₂.A[2]+A⁰))) 
        - u⁻ᵣ₋₁*(ϕ′.u⁺*cos(ϕ′.θ⁺-ϕᵣ₋₁.θ⁻+g*ϕᵣ₋₁.A[1]) - ϕ.u⁺*cos(ϕ.θ⁺-ϕᵣ₋₁.θ⁻+g*ϕᵣ₋₁.A[1]))
        + u⁻ᵣ₊₂*(ϕ′.u⁺*cos(ϕᵣ₊₂.θ⁻-ϕ′.θ⁺+g*(ϕ′.A[2]+A⁰)) - ϕ.u⁺*cos(ϕᵣ₊₂.θ⁻-ϕ.θ⁺+g*(ϕ.A[2]+A⁰))) 
        - u⁻ᵣ₊₁*(ϕ′.u⁺*cos(ϕᵣ₊₁.θ⁻-ϕ′.θ⁺+g*ϕ′.A[1]) - ϕ.u⁺*cos(ϕᵣ₊₁.θ⁻-ϕ.θ⁺+g*ϕ.A[1])) 
        + ϕᵣ₋₂.u⁺*(u′⁻*cos(ϕ′.θ⁻-ϕᵣ₋₂.θ⁺+g*(ϕᵣ₋₂.A[2]+A⁰)) - u⁻*cos(ϕ.θ⁻-ϕᵣ₋₂.θ⁺+g*(ϕᵣ₋₂.A[2]+A⁰))) 
        - ϕᵣ₋₁.u⁺*(u′⁻*cos(ϕ′.θ⁻-ϕᵣ₋₁.θ⁺+g*ϕᵣ₋₁.A[1]) - u⁻*cos(ϕ.θ⁻-ϕᵣ₋₁.θ⁺+g*ϕᵣ₋₁.A[1])))
    
    # Mixed gradient terms
    δE += ψ.γ^2*(ψ.ν-1)*(ϕᵣ₊₂.u⁺*u⁻ᵣ₊₁*(sin(ϕᵣ₊₁.θ⁻-ϕᵣ₊₂.θ⁺ + g*(ϕ′.A[1] - (ϕ′.A[2]+A⁰))) 
            - sin(ϕᵣ₊₁.θ⁻-ϕᵣ₊₂.θ⁺ + g*(ϕ.A[1] - (ϕ.A[2]+A⁰)))) 
        -ϕᵣ₊₂.u⁺*(u′⁻*sin(ϕᵣ₊₂.θ⁺-ϕ′.θ⁻+g*(ϕ′.A[2]+A⁰)) - u⁻*sin(ϕᵣ₊₂.θ⁺-ϕ.θ⁻+g*(ϕ.A[2]+A⁰))) 
        +ϕᵣ₊₁.u⁺*(u′⁻*sin(ϕᵣ₊₁.θ⁺-ϕ′.θ⁻+g*ϕ′.A[1]) - u⁻*sin(ϕᵣ₊₁.θ⁺-ϕ.θ⁻+g*ϕ.A[1])) 
        +u⁻ᵣ₋₂₊₁*(ϕ′.u⁺*sin(ϕᵣ₋₂₊₁.θ⁻-ϕ′.θ⁺+g*(ϕᵣ₋₂.A[1] - (ϕᵣ₋₂.A[2]+A⁰))) 
            - ϕ.u⁺*sin(ϕᵣ₋₂₊₁.θ⁻-ϕ.θ⁺+g*(ϕᵣ₋₂.A[1] - (ϕᵣ₋₂.A[2]+A⁰)))) 
        +ϕᵣ₋₁₊₂.u⁺*(u′⁻*sin(ϕ′.θ⁻-ϕᵣ₋₁₊₂.θ⁺+g*(ϕᵣ₋₁.A[1]-(ϕᵣ₋₁.A[2]+A⁰₋))) 
            - u⁻*sin(ϕ.θ⁻-ϕᵣ₋₁₊₂.θ⁺+g*(ϕᵣ₋₁.A[1]-(ϕᵣ₋₁.A[2]+A⁰₋)))) 
        -u⁻ᵣ₋₂*(ϕ′.u⁺*sin(ϕ′.θ⁺-ϕᵣ₋₂.θ⁻+g*(ϕᵣ₋₂.A[2]+A⁰)) - ϕ.u⁺*sin(ϕ.θ⁺-ϕᵣ₋₂.θ⁻+g*(ϕᵣ₋₂.A[2]+A⁰))) 
        +u⁻ᵣ₋₁*(ϕ′.u⁺*sin(ϕ′.θ⁺-ϕᵣ₋₁.θ⁻+g*ϕᵣ₋₁.A[1]) - ϕ.u⁺*sin(ϕ.θ⁺-ϕᵣ₋₁.θ⁻+g*ϕᵣ₋₁.A[1])) 
        -(u⁻ᵣ₊₂*ϕᵣ₊₁.u⁺*(sin(ϕᵣ₊₁.θ⁺-ϕᵣ₊₂.θ⁻ + g*(ϕ′.A[1] - (ϕ′.A[2]+A⁰))) 
            - sin(ϕᵣ₊₁.θ⁺-ϕᵣ₊₂.θ⁻ + g*(ϕ.A[1] - (ϕ.A[2]+A⁰)))) 
        -u⁻ᵣ₊₂*(ϕ′.u⁺*sin(ϕᵣ₊₂.θ⁻-ϕ′.θ⁺+g*(ϕ′.A[2]+A⁰)) - ϕ.u⁺*sin(ϕᵣ₊₂.θ⁻-ϕ.θ⁺+g*(ϕ.A[2]+A⁰))) 
        +u⁻ᵣ₊₁*(ϕ′.u⁺*sin(ϕᵣ₊₁.θ⁻-ϕ′.θ⁺+g*ϕ′.A[1]) - ϕ.u⁺*sin(ϕᵣ₊₁.θ⁻-ϕ.θ⁺+g*ϕ.A[1])) 
        +ϕᵣ₋₂₊₁.u⁺*(u′⁻*sin(ϕᵣ₋₂₊₁.θ⁺-ϕ′.θ⁻+g*(ϕᵣ₋₂.A[1] - (ϕᵣ₋₂.A[2]+A⁰))) 
            - u⁻*sin(ϕᵣ₋₂₊₁.θ⁺-ϕ.θ⁻+g*(ϕᵣ₋₂.A[1] - (ϕᵣ₋₂.A[2]+A⁰)))) 
        +u⁻ᵣ₋₁₊₂*(ϕ′.u⁺*sin(ϕ′.θ⁺-ϕᵣ₋₁₊₂.θ⁻+g*(ϕᵣ₋₁.A[1]-(ϕᵣ₋₁.A[2]+A⁰₋))) 
            - ϕ.u⁺*sin(ϕ.θ⁺-ϕᵣ₋₁₊₂.θ⁻+g*(ϕᵣ₋₁.A[1]-(ϕᵣ₋₁.A[2]+A⁰₋)))) 
        -ϕᵣ₋₂.u⁺*(u′⁻*sin(ϕ′.θ⁻-ϕᵣ₋₂.θ⁺+g*(ϕᵣ₋₂.A[2]+A⁰)) - u⁻*sin(ϕ.θ⁻-ϕᵣ₋₂.θ⁺+g*(ϕᵣ₋₂.A[2]+A⁰))) 
        +ϕᵣ₋₁.u⁺*(u′⁻*sin(ϕ′.θ⁻-ϕᵣ₋₁.θ⁺+g*ϕᵣ₋₁.A[1]) - u⁻*sin(ϕ.θ⁻-ϕᵣ₋₁.θ⁺+g*ϕᵣ₋₁.A[1])))
        +2*(ϕ′.u⁺*u′⁻*sin(ϕ′.θ⁻-ϕ′.θ⁺) - ϕ.u⁺*u⁻*sin(ϕ.θ⁻-ϕ.θ⁺)))
    
    # Then calculate the Gauge field contribution
    # First contribution from current position
    δE += (ϕ′.A[1] + ϕᵣ₊₁.A[2] - ϕᵣ₊₂.A[1] - ϕ′.A[2])^2 - (ϕ.A[1] + ϕᵣ₊₁.A[2] - ϕᵣ₊₂.A[1] - ϕ.A[2])^2
    # Then from position r-x
    δE += (ϕᵣ₋₁.A[1] + ϕ′.A[2] - ϕᵣ₋₁₊₂.A[1] - ϕᵣ₋₁.A[2])^2 - (ϕᵣ₋₁.A[1] + ϕ.A[2] - ϕᵣ₋₁₊₂.A[1] - ϕᵣ₋₁.A[2])^2
    # Then from position r-y
    δE += (ϕᵣ₋₂.A[1] + ϕᵣ₋₂₊₁.A[2] - ϕ′.A[1] - ϕᵣ₋₂.A[2])^2 - (ϕᵣ₋₂.A[1] + ϕᵣ₋₂₊₁.A[2] - ϕ.A[1] - ϕᵣ₋₂.A[2])^2
end

ΔE3 (generic function with 1 method)

In [6]:
# Test case. We make a 3x3 lattice, and put two different lattice sites in the middle
# i.e. the [2,2] position. Then we use the different functions to calculate the energy
# difference associated with this change.

ψ₂ = initializeState(3,2)
site = LatticeSite([0.0,0.0], 0, 0,1)
@show Ly = size(ψ₂.lattice,1)
ψ₁ = copy(ψ₂)
ψ₁.lattice[2,2] = copy(site)
@show ΔE2(ψ₂,site,[2,2])
@show ΔE3(ψ₂,site,[2,2])
@show E(ψ₁)-E(ψ₂)
@show ΔE2(ψ₂,site,[2,2]) == ΔE3(ψ₂,site,[2,2])
@show E(ψ₁)
@show isapprox(E(ψ₁)-E(ψ₂), ΔE3(ψ₂,site,[2,2]); atol=0.00000000000001e6, rtol=0)

Ly = size(ψ₂.lattice, 1) = 3
ΔE2(ψ₂, site, [2, 2]) = -1.1743661188699056e6
ΔE3(ψ₂, site, [2, 2]) = -1.1743661188699056e6
E(ψ₁) - E(ψ₂) = -1.1743661188699054e6
ΔE2(ψ₂, site, [2, 2]) == ΔE3(ψ₂, site, [2, 2]) = true
E(ψ₁) = 4.754825752254953e6
isapprox(E(ψ₁) - E(ψ₂), ΔE3(ψ₂, site, [2, 2]); atol=1.0e-8, rtol=0) = true


true

It seems that the difference between using `E(ψ₁)-E(ψ₂)`and `ΔE2(ψ₂,site,[2,2])` is floating point error.

In [7]:
using BenchmarkTools
@benchmark ΔE2(ψ₂,site,[2,2])

BenchmarkTools.Trial: 
  memory estimate:  112 bytes
  allocs estimate:  2
  --------------
  minimum time:     1.789 μs (0.00% GC)
  median time:      1.812 μs (0.00% GC)
  mean time:        1.855 μs (0.00% GC)
  maximum time:     4.558 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     10

In [8]:
@benchmark ΔE3(ψ₂,site,[2,2])

BenchmarkTools.Trial: 
  memory estimate:  224 bytes
  allocs estimate:  9
  --------------
  minimum time:     1.298 μs (0.00% GC)
  median time:      1.317 μs (0.00% GC)
  mean time:        1.400 μs (1.95% GC)
  maximum time:     281.364 μs (96.88% GC)
  --------------
  samples:          10000
  evals/sample:     10

For some reason there seems to be a problem with `ΔE3` allocating memory to heap, but still it is faster than `ΔE2` because we are not wasting time summing 0es. Not sure how to prevent the memory issue

# Proposal of a local state update
Given a state $\Psi_i$ we want to propose a new state $\Psi_{i+1}$ that can later be accepted according to the Metropolis-Hastings algorithm. The proposal function thus has to take in a given state $\Psi_i$ and give out a new
state $\Psi_{i+1}$ as well as the energy difference between the two states since this can be effectively calculated depending on what kind of update is done to the state. Thus we don't need to call $E(\Psi)$ twice to find the difference in energy.

The local update is done at a particular lattice site where all degrees of freedom are updated, thus the function also has to know what lattice position to update.

Remember that the proposal probability for a new state $P(\Psi_i\to\Psi_j)$ has to be equal to the symmetric update

$$P(\Psi_i\to\Psi_j) = P(\Psi_j\to\Psi_i),$$

for the update to fulfill detailed balance.

So far we do a very simple thing of defining a fixed interval around the original value, from within which the new value is uniformly randomly picked. Given a value $\theta^+_\mathbf{r}$ at the lattice site $\mathbf{r}$, the new value 
$\theta'{}^+_\mathbf{r}$ is thus picked uniformly from the interval 
$$\theta'{}^+_\mathbf{r}\in(\theta^+_\mathbf{r}-\theta_\text{max},\;\theta^+_\mathbf{r}+\theta_\text{max})$$

In [13]:
Pkg.add("Distributions")
using Distributions

INFO: Cloning cache of Calculus from https://github.com/JuliaMath/Calculus.jl.git
INFO: Cloning cache of Distributions from https://github.com/JuliaStats/Distributions.jl.git
INFO: Cloning cache of PDMats from https://github.com/JuliaStats/PDMats.jl.git
INFO: Cloning cache of QuadGK from https://github.com/JuliaMath/QuadGK.jl.git
INFO: Cloning cache of Rmath from https://github.com/JuliaStats/Rmath.jl.git
INFO: Cloning cache of SpecialFunctions from https://github.com/JuliaMath/SpecialFunctions.jl.git
INFO: Cloning cache of StatsFuns from https://github.com/JuliaStats/StatsFuns.jl.git
INFO: Installing Calculus v0.4.0
INFO: Installing Distributions v0.15.0
INFO: Installing PDMats v0.8.0
INFO: Installing QuadGK v0.2.1
INFO: Installing Rmath v0.4.0
INFO: Installing SpecialFunctions v0.6.0
INFO: Installing StatsFuns v0.6.0
INFO: Building SpecialFunctions
INFO: Building Rmath
Info: Downloading https://github.com/staticfloat/RmathBuilder/releases/download/v0.2.0-1/libRmath.x86_64-linux-gnu.t

[20:47:32] ######################################################################### 100.0%[20:47:30] ##O=#  #                                                                       [20:47:30] #=#=-#  #                                                                      [20:47:30] -#O#- #   #                                                                    [20:47:30] -=#=#   #   #                                                                  [20:47:30] -=O#- #  #    #                                                                [20:47:31] -=O=#  #   #    #                                                              [20:47:31] [20:47:31] #########                                                                  13.3%[20:47:31] ###################                                                        27.1%[20:47:31] ####################                                                       28.0%[20:47:31] ########################################                                  

INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of Distributions
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Precompiling module Distributions.


In [16]:
rand(Uniform(-1,1))

0.1364828401458298

In [17]:
function proposeLocalUpdate(ψᵢ::State, r::Array{Int64, 1})
    const θmax = 3/100 # How far away from the original value, the proposed new value for θ should be.
    const umax = 3/100 # How far away from the original value, the proposed amplitude u should be.
    const Amax = 3/100
    const x = r[1]
    const y = r[2]
    (size(ψᵢ.lattice,1) < y || size(ψᵢ.lattice,2) < x) && throw(DomainError()) # r is outside the range of the lattice.
    
    # Save the lattice site at position r in a separate variable s.
    s = ψᵢ.lattice[y,x]
    
    # Construct new configuration at lattice site.
    newLatticeSite = LatticeSite([s.A[1]+rand(Uniform(-Amax,Amax)), s.A[2]+rand(Uniform(-Amax,Amax))],
        mod(s.θ⁺ + rand(Uniform(-θmax,θmax)), 2π), mod(s.θ⁻ + rand(Uniform(-θmax,θmax)), 2π), 
        mod(s.u⁺ + rand(Uniform(-umx,umax)),1))
    
    # Calculate energy difference between the two configurations and return this with the proposed lattice site.
    (newLatticeSite, ΔE3(ψᵢ,newLatticeSite,r))
end    

proposeLocalUpdate (generic function with 1 method)

# Metropolis Hastings update

Now we create a function that implements the acceptance step of the Metropolis Hastings algorithm.
1. From state $\Psi_i$, propose a new state $\Psi_{i+1}$
2. Calculate $\Delta E_{i,i+1}$.
3. Generate a random number r between $(0,1]$.
4. If $r\leq e^{-\beta\Delta E_{i,i+1}}$, accept the state

Since the 
> rand() 

function creates a random number in $[0,1)$ we create a random number in $(0,1]$ by mapping $0\mapsto1$.

In [10]:
function metropolisHastingUpdate!(ψ::State, r::Array{Int64,1}, β::Float64)
    (site, δE) = proposeLocalUpdate(ψ,r)
    
    # Create random number ran ∈ (0,1].
    ran = rand()
    if ran==0
        ran=1
    end
    
    # Update state with probability min(1, e^{-β⋅δE})
    # and return the energy of final state regardless of whether it gets updated or not.
    if log(ran) <= -β*δE
        ψ.lattice[r...] = site
        δE
    else
        δE = 0
    end
end

metropolisHastingUpdate! (generic function with 1 method)

# Monte Carlo sweep

Try to update the entire lattice once.

In [11]:
function mcSweep!(ψ::State, β::Float64)
    
    # Find size of the lattice L
    L = size(ψ.lattice,1)
    
    # Do a Metropolis-Hastings update for all lattice sites on the lattice.
    for x in 1:L, y in 1:L
        metropolisHastingUpdate!(ψ, [x,y], β)
    end
end

mcSweep! (generic function with 1 method)

# Main MC measurements function

This is the function that will do all the work. Here we call all the other functions we have created.

After $M_\text{th}$ Monte-Carlo sweeps (MCS) we do a measurement. Before doing any measurements we wait
$M_\text{skip}$ MCS for the system to reach equilibrium. We do $M$ measurements. Let the lattice be of size $N\times N$,
the magnetic filling fraction is $f$ while $\beta$ is the inverse temperature.

In [12]:
function mcMeasure(M::Int64, Mth::Int64, Mskip::Int64, N::Int64, f::Float64, β::Float64)
    ψ = initializeState(N, f, 2)
    
    # As an example we just want to measure the energies of the states.
    e = zeros(M)
    
    # Thermalize the system
    for i=1:Mskip
        mcSweep!(ψ,β)
    end
    
    # Then do M measurements
    for i=1:(M-1)
        e[i] = E(ψ)
        for j=1:Mth-1
            mcSweep!(ψ,β)
        end
    end
    e[M] = E(ψ)
    
    # Plot results
    plot(1:M, e)
end 

mcMeasure (generic function with 1 method)